In [1]:
# Import necessary libraries
import cv2
import numpy as np

In [2]:
# Function to swap faces
def swap_faces(source_face, target_face):
    # Resize the source face to match the dimensions of the target face
    source_face_resized = cv2.resize(source_face, (target_face.shape[1], target_face.shape[0]))
    return source_face_resized



In [3]:
def main():
    # Load the pre-trained Haar cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Open a connection to the webcam (change the argument to 0 for built-in webcam)
    cap = cv2.VideoCapture(1)
    
    # Read source face image
    source_face = cv2.imread('F:/code/sourceimages/source_face.jpg')
    
    if source_face is None:
        print("Source face image not found.")
        return
    
    while True:
        ret, frame = cap.read()  # Capture a frame from the webcam
        
        if not ret:
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        for (x_t, y_t, w_t, h_t) in faces:
            # Extract the target face region
            target_face = frame[y_t:y_t+h_t, x_t:x_t+w_t]
            
            if target_face.shape[0] > 0 and target_face.shape[1] > 0:
                # Replace the target face with the source face
                swapped_face = swap_faces(source_face, target_face)
                frame[y_t:y_t+h_t, x_t:x_t+w_t] = swapped_face
        
        # Create a mask for the faces
        mask = np.zeros_like(frame)
        for (x, y, w, h) in faces:
            cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 255, 255), -1)
        
        # Apply Gaussian blur to the background using the mask
        blurred_frame = cv2.GaussianBlur(frame, (21, 21), 0)
        frame = cv2.bitwise_and(frame, mask)
        frame = cv2.add(frame, blurred_frame)
        
        # Display the frame with swapped faces and blurred background
        cv2.imshow("Live Face Swapping with Blurred Background", frame)
        
        # Break the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
